## Gate cutting to reduce circuit width

In [1]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import PauliList
from qiskit_aer.primitives import Sampler

from circuit_knitting.cutting import partition_problem, reconstruct_expectation_values

Specify a circuit and some observables

In [2]:
circuit = EfficientSU2(4, entanglement="linear", reps=2).decompose()
circuit.assign_parameters([0.4] * len(circuit.parameters), inplace=True)
observables = PauliList(["ZZII", "IZZI", "IIZZ", "XIXI", "ZIZZ", "IXIX"])

Cut the circuit and observables by using `partition_problem` and specifying qubit partitions

In [3]:
partitioned_problem = partition_problem(
    circuit=circuit,
    partition_labels="AABB",
    num_samples=np.inf,
    observables=observables,
)
partitioned_problem._fields

('subcircuits', 'subexperiments', 'weights', 'bases', 'subobservables')

Run the subexperiments for each partition using the Qiskit Aer primitives.

In [4]:
# Set up Qiskit Aer Sampler primitives.
sampler_a = Sampler(run_options={"shots": 2**12})
sampler_b = Sampler(run_options={"shots": 2**12})

# Retrieve the quasi-probability distributions for each subexperiment
quasi_dists_a = (
    sampler_a.run(partitioned_problem.subexperiments["A"]).result().quasi_dists
)
quasi_dists_b = (
    sampler_b.run(partitioned_problem.subexperiments["B"]).result().quasi_dists
)

To run using Qiskit Runtime primitives, rather than Qiskit Aer, replace the code above with this commented block.

In [5]:
# from qiskit_ibm_runtime import Session, Options, Sampler

# with Session(backend="<BACKEND_NAME>") as session:
#    options = Options()
#    options.execution.shots = 2**14
#    sampler_a = Sampler(options=options)
#    sampler_b = Sampler(options=options)

#    job_a = sampler_a.run(partitioned_problem.subexperiments["A"])
#    job_b = sampler_b.run(partitioned_problem.subexperiments["B"])

#    quasi_dists_a = job_a.result().quasi_dists
#    quasi_dists_b = job_b.result().quasi_dists

#    session.close()

Reconstruct the full expectation value, given the results of cutting

In [6]:
quasi_dists = {"A": quasi_dists_a, "B": quasi_dists_b}
simulated_expvals = reconstruct_expectation_values(
    partitioned_problem.subexperiments,
    partitioned_problem.subobservables,
    partitioned_problem.weights,
    quasi_dists,
)
simulated_expvals

[0.3190186619758605,
 0.5264472365379331,
 0.5636612176895139,
 0.11103600263595576,
 0.2603829503059386,
 -0.1568573713302612]